In [1]:
import pandas as pd
import openpyxl

In [131]:
# pip install openpyxl pandas

In [372]:
threat_counts

Подвергались ли вы в 2022 году физическому насилию из-за сексуальной ориентации и/или трансгендерности?  \
Федеральный округ                                                                                                                                  
Дальневосточный федеральный округ   Нет                                                184                                                         
                                    Да                                                  18                                                         
Приволжский федеральный округ       Нет                                                830                                                         
                                    Да                                                  69                                                         
Северо-Западный федеральный округ   Нет                                               1504                                                         
                                    Да                                                  92                                                         
Северо-Кавказский федеральный округ Нет                                                 52                                                         
                                    Да                                                   7                                                         
Сибирский федеральный округ         Нет                                                538                                                         
                                    Да                                                  61                                                         
Уральский федеральный округ         Нет                                                470                                                         
                                    Да                                                  47                                                         
Центральный федеральный округ       Нет                                               2123                                                         
                                    Да                                                 165                                                         
Южный федеральный округ             Нет                                                262                                                         
                                    Да                                                  17                                                         

                                         Подвергались ли вы в 2022 году сексуализированному насилию (в том числе домогательствам и попыткам изнасилования) из-за вашей сексуальной ориентации и/или трансгендерности?  \
Федеральный округ                                                                                                                                                                                                       
Дальневосточный федеральный округ   Нет                                                189                                                                                                                              
                                    Да                                                  13                                                                                                                              
Приволжский федеральный округ       Нет                                                830                                                                                                                              
                                    Да                                                  69                                                                                                                              
Северо-Западный федеральный округ   Нет                                   

In [375]:

df = pd.read_excel('2022_raw.xlsx', engine='openpyxl')

#selec threat related colums
df_threat = df.iloc[:, [7, 8] + list(range(32, 36))+list(range(37, 42))]
df_threat = df_threat.drop(columns=df_threat.columns[7])
# df_threat.head(1)

#coun frequencies
df_threat_counts = {}
first_col = df_threat.columns[1] ##Федеральный округ
for col in df_threat.columns[2:]:
    df_threat_counts[col] = df_threat.groupby(first_col)[col].value_counts()#.fillna(0)
#     print(df_threat_counts[col])
# pd.DataFrame(result)

# transform to data frame
threat_counts=pd.DataFrame(df_threat_counts)
threat_counts['all'] = threat_counts.iloc[:, 2:].sum(axis=1)

districts = df_threat[first_col].unique()
district_percentages = {}
district_threat_percentages = {}
district_threat_count = {}
district_count = {}
distrcict_all={}
all_districts_yes = [0] * len(threat_counts.columns)
all_districts_no = [0] * len(threat_counts.columns)

# calculate percentages
for district in districts:
    total_yes = threat_counts.loc[(district, 'Да'), 'all'] if (district, 'Да') in threat_counts.index else 0
    total_no = threat_counts.loc[(district, 'Нет'), 'all'] if (district, 'Нет') in threat_counts.index else 0

#     print(total_yes)
#     print(total_no)
    if total_yes + total_no != 0: 
        district_percentages[district] =  (total_yes / (total_yes + total_no)) * 100
    else:
        district_percentages[district] = 0

    for col in threat_counts.columns:
        da_count = threat_counts.loc[(district, 'Да'), col] if (district, 'Да') in threat_counts.index else 0
        net_count = threat_counts.loc[(district, 'Нет'), col] if (district, 'Нет') in threat_counts.index else 0
        
#         print(threat_counts.loc[(district, 'Да')])
#         print(da_count)
#         print(net_count)
#         print(threat_counts.loc[(district, 'Да'), col])
        if da_count + net_count != 0: 
            district_threat_percentages[(district, col)] = (da_count / (da_count + net_count)) * 100
        elif (district, 'Да') not in data.index:
            district_threat_percentages[(district, col)] = 0
           
        else:
            district_threat_percentages[(district, col)] = 100

#     print(data.loc[(district, 'Да')])
    all_districts_yes += threat_counts.loc[(district, 'Да')].values #if (district, 'Да') in data.index else [0] * len(data.columns)
    all_districts_no += threat_counts.loc[(district, 'Нет')].values #if (district, 'Нет') in data.index else [0] * len(data.columns)

    
# transform to data frame
df_district_percentages = pd.DataFrame(list(district_percentages.items()), columns=['District', 'Percentage'])
df_district_threat_percentages = pd.DataFrame(list(district_threat_percentages.items()), columns=['District_Column', 'Percentage'])
df_district_threat_percentages[['District', 'Column']] = pd.DataFrame(df_district_threat_percentages['District_Column'].tolist(), index=df_district_threat_percentages.index)
df_district_threat_percentages = df_district_threat_percentages.pivot(index='District', columns='Column', values='Percentage')


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [376]:
threat_percentages = all_districts_yes/(all_districts_yes+all_districts_no)*100
new_row = pd.Series(threat_percentages, index=df_district_threat_percentages.columns, name="All Districts")
df_district_threat_percentages = df_district_threat_percentages.append(new_row)

/var/folders/_b/xz5gy50j1vbb665n4v6n1z4c0000gn/T/ipykernel_89373/2916998733.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_district_threat_percentages = df_district_threat_percentages.append(new_row)


In [379]:
# df_district_threat_percentages

In [371]:
# df_district_percentages

In [378]:
df_district_threat_percentages.to_csv('summary_threat_percent.csv')
# df_district_percentages.to_csv('summary_threat_percent_total.csv')



In [210]:
df.columns[28:31]
df.columns[51:53] #52 to threats
df.columns[55:57]
df.columns[57:59]
df.columns[63]
df.columns[65]

'Сталкивались ли вы в 2022 году с трудностями при взаимодействии с детскими образовательными или медицинскими учреждениями из-за ваших сексуальной ориентации и/или трансгендерности?'

In [ ]:

df_discrim = df.iloc[:, [7,28,30]+list(range(51,53))+list(range(55,57))+list(range(57,59))+[63,67] ]

In [182]:
df.columns

Index(['Отметка времени',
       'Как вы себя определяете (ваша гендерная идентичность)?',
       'Идентифицируете ли вы себя как трансгендерного человека? ',
       'Ваша сексуальная ориентация', 'Являетесь ли вы интерсекс-человеком?',
       'Ваш возраст', 'Где вы проживаете? ',
       'Выберите, пожалуйста, регион, в котором вы проживаете',
       'Федеральный округ',
       'Кто из ваших друзей знает о том, что вы ЛГБТК+ персона?',
       'Кто из ваших родных знает о том, что вы ЛГБТК+ персона?',
       'Кто из ваших одноклассников/однокурсников/коллег (в нынешнем учебном/рабочем коллективе) знает о том, что вы ЛГБТК+ персона?',
       'Ваш круг общения, в основном, состоит из...',
       'Ваш круг общения – это, в основном, (по вашим ощущениям)…',
       'Ваше образование',
       'Охарактеризуйте, пожалуйста, ваше общее материальное положение.',
       'Где вы преимущественно жили в 2022 году?',
       'Есть ли у вас иждивенцы - те, кто не имеет собственного источника дохода и ко

In [222]:

df = pd.read_excel('2022_raw.xlsx', engine='openpyxl')

#selec threat related colums
df_discrim = df.iloc[:, [7,28,30]+list(range(51,53))+list(range(55,57))+list(range(57,59))+[63,67] ]
# df_threat.head(1)
# df.columns[28:31]
# df.columns[51:53] #52 to threats
# df.columns[55:57]
# df.columns[57:59]
# df.columns[63]
# df.columns[65]

#coun frequencies
result = {}
first_col = df_discrim.columns[1] ##Федеральный округ
for col in df_discrim.columns[2:]:
    result[col] = df_discrim.groupby(first_col)[col].value_counts().fillna(0)

pd.DataFrame(result)

# # transform to data frame
# data=pd.DataFrame(result)
# data['all'] = data.iloc[:, 2:].sum(axis=1)

# districts = df_threat[first_col].unique()
# district_percentages = {}
# district_threat_percentages = {}
# district_threat_count = {}
# district_count = {}
# # calculate percentages
# for district in districts:
#     total_yes = data.loc[(district, 'Да'), 'all']
#     total_no = data.loc[(district, 'Нет'), 'all']
#     district_percentages[district] =  (total_yes / (total_yes + total_no)) * 100
 
    
# #     district_threat_percentages[district] = data.loc[(district, 'Да')]/(data.loc[(district, 'Да')]+data.loc[(district,'Нет')])*100

#     for col in data.columns:
#         if (district, 'Да') in data.index and (district, 'Нет') in data.index:
#             district_threat_percentages[(district, col)] = data.loc[(district, 'Да'), col] / (data.loc[(district, 'Да'), col] + data.loc[(district, 'Нет'), col]) * 100
#         else:
#             district_threat_percentages[(district, col)] = None

            

# # transform to data frame
# df_district_percentages = pd.DataFrame(list(district_percentages.items()), columns=['District', 'Percentage'])
# df_district_threat_percentages = pd.DataFrame(list(district_threat_percentages.items()), columns=['District_Column', 'Percentage'])
# df_district_threat_percentages[['District', 'Column']] = pd.DataFrame(df_district_threat_percentages['District_Column'].tolist(), index=df_district_threat_percentages.index)
# df_district_threat_percentages = df_district_threat_percentages.pivot(index='District', columns='Column', values='Percentage')


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Приходилось ли вам в 2022 году терять работу/место учёбы из-за сексуальной ориентации и/или трансгендерности?  \
Были ли у вас в 2022 году какие-либо проблемы п...                                                                                                                                                                     
Да                                                 \nМой ответ: https://docs.google.com/document/d...                                                NaN                                                               
                                                   -                                                                                                 NaN                                                               
                                                   .                                                                                                 NaN                                                               
                                                   2020, отказ в замене водительского удостоверения                                                  NaN                                                               
                                                   2021 году полицию обращено было на чает угроза ...                                                NaN                                                               
...                                                                                                                                                  ...                                                               
Нет                                                уехал из России в Польшу, бросив все                                                              NaN                                                               
                                                   я не могла заключить брак в россии, пришлось эт...                                                NaN                                                               
                                                   я пока не понял                                                                                   NaN                                                               
                                                   я потеряла надежду на безопасность лгбт персон ...                                                NaN                                                               
                                                   я приспособилась и стараюсь особо не ныть из-за...                                                NaN                                                               

                                                                                                       Приходилось ли вам в 2022 году сталкиваться с отказом в предоставлении медицинских услуг из-за вашей сексуальной ориентации и/или трансгендерностии?  \
Были ли у вас в 2022 году какие-либо проблемы п...                                                                                                                                                                                                            
Да                                                 \nМой ответ: https://docs.google.com/document/d...                                                NaN                                                                                                      
                                                   -                                                                                                 NaN                                                                                                      
                                                   .                                                                                                 NaN                                                                                                      
                   

In [221]:
df_discrim.loc[28:31]

,"Выберите, пожалуйста, регион, в котором вы проживаете",Были ли у вас в 2022 году какие-либо проблемы при поиске места работы/учебы из-за сексуальной ориентации и/или трансгендерности?,Сталкивались ли вы с каким-либо давлением на работе/учёбе из-за сексуальной ориентации и/или трансгендерности в 2022 году?,Приходилось ли вам в 2022 году терять работу/место учёбы из-за сексуальной ориентации и/или трансгендерности?,Приходилось ли вам в 2022 году сталкиваться с отказом в предоставлении медицинских услуг из-за вашей сексуальной ориентации и/или трансгендерностии?,Приходилось ли вам в 2022 году сталкиваться с насильственными/ дискриминирующими действиями врачей на почве гомофобии/трансфобии?,Приходилось ли вам в 2022 году сталкиваться со случаями отказа в доступе к государственным услугам из-за сексуальной ориентации и/или трансгендерности?,"Если вы сталкивались с ограничением в сфере услуг, которое не описано выше, или готовы рассказать о вашем опыте подробнее, вы можете написать об этом здесь (пожалуйста, уточните, когда это произошло):","Сталкивались ли вы в 2022 году с ситуацией, когда вас лишили жилья из-за вашей сексуальной ориентации и/или трансгендерности?","Сталкивались ли вы в 2022 году с трудностями в поиске жилья в связи с вашей сексуальной ориентацией, трансгендерностью или гендерной неконформностью?",Сталкивались ли вы в 2022 году с трудностями при взаимодействии с органами опеки из-за ваших сексуальной ориентации и/или трансгендерности?,Повлиял ли на вас в целом вооруженный конфликт между Россией и Украиной в 2022 году?
28,ОРЛОВСКАЯ ОБЛАСТЬ,Да,Да,Затрудняюсь ответить,NaN,NaN,Да,"Отказ продовать одежду и обувь в магазине, кот...",Нет,Нет,NaN,"Стало страшно, здоровье ухудшилось, дискримина..."
29,РЕСПУБЛИКА ТАТАРСТАН,NaN,Да,Нет,NaN,NaN,Нет,NaN,Нет,Нет,NaN,Нет
30,МОСКОВСКАЯ ОБЛАСТЬ,Да,Да,Нет,NaN,NaN,Нет,NaN,Нет,Нет,NaN,Нет
31,КРАСНОДАРСКИЙ КРАЙ,NaN,Да,Нет,Нет,Нет,Нет,NaN,Нет,Нет,NaN,Да


In [214]:
result

{'Сталкивались ли вы с каким-либо давлением на работе/учёбе из-за сексуальной ориентации и/или трансгендерности в 2022 году?': Были ли у вас в 2022 году какие-либо проблемы при поиске места работы/учебы из-за сексуальной ориентации и/или трансгендерности?  Сталкивались ли вы с каким-либо давлением на работе/учёбе из-за сексуальной ориентации и/или трансгендерности в 2022 году?
 Да                                                                                                                                Да                                                                                                                             434
                                                                                                                                   Нет                                                                                                                            185
 Нет                                                                                            

In [167]:

df = pd.read_excel('2022_raw.xlsx', engine='openpyxl')

#selec threat related colums
df_threat = df.iloc[:, [7, 8] + list(range(32, 36))+list(range(37, 42))]
# df_threat.head(1)

#coun frequencies
result = {}
first_col = df_threat.columns[1] ##Федеральный округ
for col in df_threat.columns[2:]:
    result[col] = df_threat.groupby(first_col)[col].value_counts().fillna(0)

# pd.DataFrame(result)

# transform to data frame
data=pd.DataFrame(result)
data['all'] = data.iloc[:, 2:].sum(axis=1)

districts = df_threat[first_col].unique()
district_percentages = {}
district_threat_percentages = {}
district_threat_count = {}
district_count = {}
# calculate percentages
for district in districts:
    total_yes = data.loc[(district, 'Да'), 'all']
    total_no = data.loc[(district, 'Нет'), 'all']
    district_percentages[district] =  (total_yes / (total_yes + total_no)) * 100
 
    
#     district_threat_percentages[district] = data.loc[(district, 'Да')]/(data.loc[(district, 'Да')]+data.loc[(district,'Нет')])*100

    for col in data.columns:
        if (district, 'Да') in data.index and (district, 'Нет') in data.index:
            district_threat_percentages[(district, col)] = data.loc[(district, 'Да'), col] / (data.loc[(district, 'Да'), col] + data.loc[(district, 'Нет'), col]) * 100
        else:
            district_threat_percentages[(district, col)] = None

            

# transform to data frame
df_district_percentages = pd.DataFrame(list(district_percentages.items()), columns=['District', 'Percentage'])
df_district_threat_percentages = pd.DataFrame(list(district_threat_percentages.items()), columns=['District_Column', 'Percentage'])
df_district_threat_percentages[['District', 'Column']] = pd.DataFrame(df_district_threat_percentages['District_Column'].tolist(), index=df_district_threat_percentages.index)
df_district_threat_percentages = df_district_threat_percentages.pivot(index='District', columns='Column', values='Percentage')


/var/folders/_b/xz5gy50j1vbb665n4v6n1z4c0000gn/T/ipykernel_89373/722702315.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(df, ignore_index=True)
/var/folders/_b/xz5gy50j1vbb665n4v6n1z4c0000gn/T/ipykernel_89373/722702315.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(df, ignore_index=True)
/var/folders/_b/xz5gy50j1vbb665n4v6n1z4c0000gn/T/ipykernel_89373/722702315.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(df, ignore_index=True)
/var/folders/_b/xz5gy50j1vbb665n4v6n1z4c0000gn/T/ipykernel_89373/722702315.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.conc

In [169]:
final_df.T

,0,1,2,3,4,5,6,7,8,9,...,150,151,152,153,154,155,156,157,158,159
index,Подвергались ли вы в 2022 году физическому нас...,Подвергались ли вы в 2022 году сексуализирован...,Сталкивались ли вы в 2022 году с угрозами физи...,Сталкивались ли вы в 2022 году с домашним наси...,Сталкивались ли вы в 2022 году со случаями при...,Сталкивались ли вы в 2022 году с грабежом по т...,Сталкивались ли вы в 2022 году с шантажом/вымо...,Сталкивались ли вы в 2022 году со случаями нез...,Сталкивались ли именно вы в 2022 году с пресле...,all,...,Подвергались ли вы в 2022 году физическому нас...,Подвергались ли вы в 2022 году сексуализирован...,Сталкивались ли вы в 2022 году с угрозами физи...,Сталкивались ли вы в 2022 году с домашним наси...,Сталкивались ли вы в 2022 году со случаями при...,Сталкивались ли вы в 2022 году с грабежом по т...,Сталкивались ли вы в 2022 году с шантажом/вымо...,Сталкивались ли вы в 2022 году со случаями нез...,Сталкивались ли именно вы в 2022 году с пресле...,all
Сибирский федеральный округ,10.183639,10.350584,27.378965,15.525876,6.343907,6.176962,17.863105,13.522538,17.529215,14.905795,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
District,Сибирский федеральный округ,Сибирский федеральный округ,Сибирский федеральный округ,Сибирский федеральный округ,Сибирский федеральный округ,Сибирский федеральный округ,Сибирский федеральный округ,Сибирский федеральный округ,Сибирский федеральный округ,Сибирский федеральный округ,...,Северо-Кавказский федеральный округ,Северо-Кавказский федеральный округ,Северо-Кавказский федеральный округ,Северо-Кавказский федеральный округ,Северо-Кавказский федеральный округ,Северо-Кавказский федеральный округ,Северо-Кавказский федеральный округ,Северо-Кавказский федеральный округ,Северо-Кавказский федеральный округ,Северо-Кавказский федеральный округ
Центральный федеральный округ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Уральский федеральный округ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Дальневосточный федеральный округ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Приволжский федеральный округ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Северо-Западный федеральный округ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Южный федеральный округ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Северо-Кавказский федеральный округ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11.864407,6.779661,22.033898,20.338983,13.559322,NaN,8.474576,13.559322,10.169492,12.590799


In [170]:
district_threat_percentages.items()

dict_items([('Сибирский федеральный округ', Подвергались ли вы в 2022 году физическому насилию из-за сексуальной ориентации и/или трансгендерности?                                                                          10.183639
Подвергались ли вы в 2022 году сексуализированному насилию (в том числе домогательствам и попыткам изнасилования) из-за вашей сексуальной ориентации и/или трансгендерности?     10.350584
Сталкивались ли вы в 2022 году с угрозами физическим насилием из-за сексуальной ориентации и/или трансгендерности?                                                               27.378965
Сталкивались ли вы в 2022 году с домашним насилием из-за сексуальной ориентации и/или трансгендерности (со стороны родителей, родственников, супругов, партнёров)?               15.525876
Сталкивались ли вы в 2022 году со случаями причинения вреда вашему имуществу, уничтожения, кражи/хищения имущества из-за вашей сексуальной ориентации и/или трансгендерности?     6.343907
Сталкивались ли вы в 

In [86]:
district_threat_percentages['Сибирский федеральный округ'].values

array([0.1133829 , 0.11545624, 0.37701149, 0.18379447, 0.06773619,
       0.0658363 , 0.21747967, 0.15637066, 0.21255061, 0.17516816])

In [95]:
df=data
district_percentages = {}
district_threat_percentages = {}
district_threat_count = {}
district_count = {}

for district in districts:
    total_yes = df.loc[(district, 'Да'), 'all']
    total_no = df.loc[(district, 'Нет'), 'all']
    district_percentages[district] = (total_yes / (total_yes + total_no)) * 100
    
    district_threat_percentages[district] = df.loc[(district, 'Да')]/df.loc[(district,'Нет')]



In [96]:
district_threat_percentages

{'Сибирский федеральный округ': Подвергались ли вы в 2022 году физическому насилию из-за сексуальной ориентации и/или трансгендерности?                                                                          0.113383
 Подвергались ли вы в 2022 году сексуализированному насилию (в том числе домогательствам и попыткам изнасилования) из-за вашей сексуальной ориентации и/или трансгендерности?     0.115456
 Сталкивались ли вы в 2022 году с угрозами физическим насилием из-за сексуальной ориентации и/или трансгендерности?                                                               0.377011
 Сталкивались ли вы в 2022 году с домашним насилием из-за сексуальной ориентации и/или трансгендерности (со стороны родителей, родственников, супругов, партнёров)?               0.183794
 Сталкивались ли вы в 2022 году со случаями причинения вреда вашему имуществу, уничтожения, кражи/хищения имущества из-за вашей сексуальной ориентации и/или трансгендерности?    0.067736
 Сталкивались ли вы в 2022 году с 

In [97]:
district_percentages=pd.DataFrame(district_percentages, index=list(data.keys())).T
# district_threat_percentages=pd.DataFrame(district_threat_percentages).T

In [98]:
district_percentages

,Подвергались ли вы в 2022 году физическому насилию из-за сексуальной ориентации и/или трансгендерности?,Подвергались ли вы в 2022 году сексуализированному насилию (в том числе домогательствам и попыткам изнасилования) из-за вашей сексуальной ориентации и/или трансгендерности?,Сталкивались ли вы в 2022 году с угрозами физическим насилием из-за сексуальной ориентации и/или трансгендерности?,"Сталкивались ли вы в 2022 году с домашним насилием из-за сексуальной ориентации и/или трансгендерности (со стороны родителей, родственников, супругов, партнёров)?","Сталкивались ли вы в 2022 году со случаями причинения вреда вашему имуществу, уничтожения, кражи/хищения имущества из-за вашей сексуальной ориентации и/или трансгендерности?",Сталкивались ли вы в 2022 году с грабежом по типу “подставных свиданий” ?,Сталкивались ли вы в 2022 году с шантажом/вымогательством с угрозой раскрытия информации о вашей сексуальной ориентации или трансгендерности?,Сталкивались ли вы в 2022 году со случаями незаконного использования ваших персональных данных из-за вашей сексуальной ориентации и/или трансгендерности?,Сталкивались ли именно вы в 2022 году с преследованиями в интернете из-за вашей сексуальной ориентации и/или трансгендерности?,all
Сибирский федеральный округ,14.905795,14.905795,14.905795,14.905795,14.905795,14.905795,14.905795,14.905795,14.905795,14.905795
Центральный федеральный округ,10.795455,10.795455,10.795455,10.795455,10.795455,10.795455,10.795455,10.795455,10.795455,10.795455
Уральский федеральный округ,14.119923,14.119923,14.119923,14.119923,14.119923,14.119923,14.119923,14.119923,14.119923,14.119923
Дальневосточный федеральный округ,15.134371,15.134371,15.134371,15.134371,15.134371,15.134371,15.134371,15.134371,15.134371,15.134371
Приволжский федеральный округ,13.062133,13.062133,13.062133,13.062133,13.062133,13.062133,13.062133,13.062133,13.062133,13.062133
Северо-Западный федеральный округ,9.559613,9.559613,9.559613,9.559613,9.559613,9.559613,9.559613,9.559613,9.559613,9.559613
Южный федеральный округ,12.288786,12.288786,12.288786,12.288786,12.288786,12.288786,12.288786,12.288786,12.288786,12.288786
Северо-Кавказский федеральный округ,12.590799,12.590799,12.590799,12.590799,12.590799,12.590799,12.590799,12.590799,12.590799,12.590799


ValueError: DataFrame constructor not properly called!

In [58]:
df.index[0][0]

'Дальневосточный федеральный округ'